## **ATIVIDADE PRÁTICA — RAG com Código de Defesa do Consumidor e LGPD**
### **1. Setup do Projeto**
### **Contexto**
Você vai construir um assistente jurídico baseado em RAG capaz de responder perguntas sobre:
- **Código de Defesa do Consumidor (CDC)**
- **Lei Geral de Proteção de Dados (LGPD)**
---
### **Estrutura do Projeto**
Crie uma pasta chamada:
rag-juridico/
Dentro dela, organize assim:
```text
rag-juridico/
 ├── dados/
 │   ├── cdc.pdf
 │   └── lgpd.pdf
 ├── ingestao.py
 ├── rag.py
 └── app.py
Requisitos
Garanta que seu ambiente tenha:
loader de PDF funcionando
embeddings configurados
ChromaDB instalado

In [ ]:
#Dependências instaladas pelo terminal do colab pip install langchain langchain-community langchain-openai chromadb pypdf tiktoken

In [ ]:
#Criação de pastas
!mkdir -p rag-juridico/dados

In [ ]:
#Verificação de pasta
!ls -R rag-juridico

rag-juridico:
dados

rag-juridico/dados:
cdc.pdf  lgpd.pdf


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("RAG_1")

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
#Teste de de leitura do Secrets
import os
print("Chave carregada:", bool(os.environ.get("OPENAI_API_KEY")))

Chave carregada: True


In [ ]:
#Visualização de onde estou na pasta
%pwd

'/content'

In [ ]:
#Comando para mudar para pasta rag-juridico
%cd /content/rag-juridico

/content/rag-juridico


In [ ]:
#Criação de arquivo ingestao.py
%%writefile ingestao.py
print("Olá, eu sou o arquivo ingestao.py")

Writing ingestao.py


In [ ]:
#leitura do arquivo
!cat ingestao.py

print("Olá, eu sou o arquivo ingestao.py")


In [ ]:
%pwd


'/content/rag-juridico'

In [ ]:
#Criação de arquivo rag.py
%%writefile rag.py
print("olá,eu sou o arquivo rag.py")


Writing rag.py


In [ ]:
#leitura do arquivo
!cat rag.py

print("olá,eu sou o arquivo rag.py")


In [ ]:
#lista do conteúdo
!ls -R /content/rag-juridico

/content/rag-juridico:
dados  ingestao.py  rag.py

/content/rag-juridico/dados:
cdc.pdf  lgpd.pdf


In [ ]:
#Criação de arquivo app.py
%%writefile app.py
print(Olá, eu sou o arquivo app.py)


Writing app.py


In [ ]:
#leitura do app.py
!cat app.py

print(Olá, eu sou o arquivo app.py)


## **2. Carregando documentos jurídicos**

### **Descrição**

Transformar os PDFs do **CDC** e da **LGPD** em documentos processáveis pelo LangChain.

***

### **Objetivo**

Carregar os arquivos PDF e convertê-los em uma lista única de `Document`, com metadados indicando a origem de cada página.

***

### **Tarefas**

1.  **Carregar os dois PDFs:**
    *   `cdc.pdf`
    *   `lgpd.pdf`

2.  **Adicionar metadados a cada página:**
    *   `fonte: "cdc"`
    *   `fonte: "lgpd"`

3.  **Imprimir ao final:**
    *   quantidade total de documentos carregados
    *   quantidade por fonte (CDC vs LGPD)

***

### **Resultado esperado**

*   Uma lista única de `Document`
*   Metadados corretamente preenchidos para cada item

In [ ]:
#Arquivos carregados /content/rag-juridico/dados/
!ls /content/rag-juridico/dados

cdc.pdf  lgpd.pdf


In [ ]:
#Arquivos originais renomeados
!ls /content/rag-juridico/dados/

cdc.pdf  lgpd.pdf


In [ ]:
#importações
import os

In [ ]:
# Loader de documentos PDF
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# Divisão de texto em blocos
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Embeddings
from langchain_openai import OpenAIEmbeddings

In [ ]:
# Banco vetorial
from langchain_community.vectorstores import Chroma

In [ ]:
# LLM
from langchain_openai import ChatOpenAI

In [ ]:
# LLM
from langchain_openai import ChatOpenAI

In [ ]:
#Verificação do caminho
import os
print(os.listdir("/content/rag-juridico/dados"))

['cdc.pdf', '.ipynb_checkpoints', 'lgpd.pdf']


In [ ]:
#Executado no terminal do google colab !pip install langchain langchain-community langchain-text-splitters chromadb sentence-transformers


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import chromadb

In [ ]:
import os

base_path = "/content/rag-juridico"

pastas = [
    "dados",
    "vectorstore"
]

for pasta in pastas:
    os.makedirs(os.path.join(base_path, pasta), exist_ok=True)

print("Estrutura criada!")

Estrutura criada!


In [ ]:
!pip -q install -U langchain langchain-community pypdf chromadb

In [ ]:

from langchain_community.document_loaders import PyPDFLoader


In [ ]:
import os

print("CWD:", os.getcwd())
print("Existe cdc? ", os.path.exists("/content/rag-juridico/dados/cdc.pdf"))
print("Existe lgpd?", os.path.exists("/content/rag-juridico/dados/lgpd.pdf"))
print("Conteúdo da pasta 'dados':", os.listdir("/content/rag-juridico/dados"))

CWD: /content/rag-juridico
Existe cdc?  True
Existe lgpd? True
Conteúdo da pasta 'dados': ['cdc.pdf', '.ipynb_checkpoints', 'lgpd.pdf']


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from collections import Counter

def carregar_pdf(caminho: str, fonte: str):
    if not isinstance(caminho, str) or not isinstance(fonte, str):
        raise TypeError("caminho e fonte devem ser strings.")
    if not os.path.exists(caminho):
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    # Carrega uma página por Document
    loader = PyPDFLoader(caminho)
    docs = loader.load()
    # Adiciona metadados de fonte
    for d in docs:
        d.metadata["fonte"] = fonte
    return docs

try:
    cdc_docs = carregar_pdf("/content/rag-juridico/dados/cdc.pdf", "cdc")
    lgpd_docs = carregar_pdf("/content/rag-juridico/dados/lgpd.pdf", "lgpd")
    documentos = cdc_docs + lgpd_docs

    print(f"Total de documentos (páginas): {len(documentos)}")
    contagem = Counter([d.metadata.get("fonte") for d in documentos])
    print("Por fonte:", dict(contagem))

    # Mostra um exemplo de metadados da primeira página
    if documentos:
        print("Exemplo de metadados da primeira página:", documentos[0].metadata)

except Exception as e:
    import traceback
    print("⚠️ Ocorreu um erro:")
    traceback.print_exc()


Total de documentos (páginas): 179
Por fonte: {'cdc': 130, 'lgpd': 49}
Exemplo de metadados da primeira página: {'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.1 (Windows)', 'creationdate': '2025-02-11T11:13:18-03:00', 'moddate': '2025-02-11T11:13:18-03:00', 'title': 'CDC_capa_2025.indd', 'trapped': '/False', 'source': '/content/rag-juridico/dados/cdc.pdf', 'total_pages': 130, 'page': 0, 'page_label': '1', 'fonte': 'cdc'}


In [ ]:
from collections import Counter

print("Total de documentos:", len(documentos))

contagem = Counter(doc.metadata["fonte"] for doc in documentos)
print("Por fonte:", contagem)

Total de documentos: 179
Por fonte: Counter({'cdc': 130, 'lgpd': 49})


## **3 — Realizando *Chunking***

### **Descrição**

Aplicar diferentes estratégias de *chunking* para analisar como elas influenciam a qualidade do RAG em textos jurídicos.

***

### **Objetivo**

Compreender o impacto de diferentes métodos de divisão textual na geração de embeddings e recuperação de informações.

***

### **Tarefas**

1.  **Criar duas funções de chunking:**
    *   Uma usando **`RecursiveCharacterTextSplitter`**
    *   Outra quebrando o texto por parágrafos  
        (usando **`CharacterTextSplitter`** com separador `\n\n`)

2.  **Gerar chunks com:**
    *   tamanho fixo entre **500–800 caracteres**
    *   *overlap* configurável (ex.: 50–150 caracteres)

3.  **Comparar os resultados:**
    *   número total de *chunks* gerados
    *   tamanho médio dos *chunks*

***

### **Pergunta para reflexão**

**Qual estratégia gera chunks mais “legíveis” para um texto jurídico?**

— Pense sobre coerência entre frases, preservação de contexto e quebra semântica.

In [ ]:
#Instalação de módulos
!pip -q install -U langchain-community

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [ ]:
!pip -q install -U langchain langchain-community langchain-text-splitters

In [ ]:
# Função de chunking usando RecursiveCharacterTextSplitter
def chunk_recursive(docs, chunk_size=700, overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )
    return splitter.split_documents(docs)

chunks_recursive = chunk_recursive(documentos)


In [ ]:
# Função de chunking usando CharacterTexteSplitter

def chunk_paragrafo(docs, chunk_size=700, overlap=100):
    splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )
    return splitter.split_documents(docs)

chunks_paragrafo = chunk_paragrafo(documentos)

In [ ]:
# Comparação
def estatisticas(chunks):
    tamanhos = [len(c.page_content) for c in chunks]
    return {
        "total_chunks": len(chunks),
        "tamanho_medio": sum(tamanhos) / len(tamanhos)
    }

print("Recursive:", estatisticas(chunks_recursive))
print("Parágrafo:", estatisticas(chunks_paragrafo))

Recursive: {'total_chunks': 512, 'tamanho_medio': 566.453125}
Parágrafo: {'total_chunks': 177, 'tamanho_medio': 1512.0225988700565}



## 💡 **Reflexão: Qual estratégia gera chunks mais “legíveis” para textos jurídicos?**

Quando dividimos um texto em *chunks* (pedaços menores), queremos que cada pedaço faça sentido sozinho. Isso é importante porque o modelo de IA vai usar esses pedaços para procurar respostas.

### 🧱 **1. Chunking Recursivo**

O método **RecursiveCharacterTextSplitter** corta o texto baseado apenas no tamanho (ex.: 700 caracteres), tentando manter a coerência quando possível.

📌 **Vantagem:**  
Funciona para qualquer tipo de texto, mesmo sem estrutura.

📌 **Desvantagem:**  
Pode cortar no meio de frases ou artigos.  
Com isso, partes da lei podem perder sentido.

📊 **Resultado obtido:**

*   **512 chunks**
*   **Tamanho médio: \~566 caracteres**

Isso significa que o texto foi muito “picado”.

***

### 📄 **2. Chunking por Parágrafos**

Aqui você divide sempre que encontrar um parágrafo (`\n\n`).  
Textos jurídicos são organizados assim:

*   artigos
*   incisos
*   alíneas
*   parágrafos

Cada parte geralmente traz uma ideia completa.

📌 **Vantagem:**  
Os chunks ficam **mais naturais e mais fáceis de entender**, porque preservam o sentido jurídico.

📌 **Desvantagem:**  
Alguns chunks podem ficar maiores do que o ideal para embeddings — mas continuam mais coerentes.

📊 **Resultado obtido:**

*   **177 chunks**
*   **Tamanho médio: \~1512 caracteres**

Ou seja, são menos chunks, porém muito mais completos.

***

## ✅ **Conclusão**

**Para textos jurídicos, a divisão por parágrafos costuma gerar chunks mais “legíveis” e mais fiéis ao significado da lei.**

Isso acontece porque:

*   cada parágrafo é uma ideia completa
*   não quebra artigos no meio
*   mantém a lógica do texto legal
*   facilita a compreensão da IA

Em outras palavras:

👉 *Chunking por parágrafos preserva o raciocínio jurídico.*  
👉 *Chunking recursivo pode quebrar o texto em pedaços pequenos demais.*


## **4 — Criando Embeddings e o Banco Vetorial**

### **Descrição**

Transformar os *chunks* jurídicos em vetores (*embeddings*) e armazená‑los em um banco vetorial persistente.

***

### **Objetivo**

Criar um repositório vetorial que permita recuperar informações relevantes a partir dos textos do **CDC** e da **LGPD**.

***

### **Tarefas**

1.  **Gerar embeddings para todos os chunks**
    *   Utilizar um modelo de embeddings compatível com LangChain.
    *   Processar todos os textos já chunkados.

2.  **Armazenar os embeddings em um vectorstore persistente**
    *   Salvar em disco para não precisar gerar novamente.
    *   O vectorstore precisa:

        *   **Recarregar os dados se já existir no diretório**
        *   **Criar do zero caso não exista**

***

### **Extra (opcional)**

Criar duas coleções separadas dentro do banco vetorial:

*   uma coleção exclusiva para o **CDC**
*   outra exclusiva para a **LGPD**

Isso permite consultas segmentadas ou comparação entre as duas leis.

In [ ]:
!pip -q install -U langchain langchain-community langchain-text-splitters chromadb


## **4 — Criando Embeddings e o Banco Vetorial**
### **Extra (opcional)**

Criar duas coleções separadas dentro do banco vetorial:

*   uma coleção exclusiva para o **CDC**
*   outra exclusiva para a **LGPD**

Isso permite consultas segmentadas ou comparação entre as duas leis.

In [ ]:
# Imports corretos
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os

# Criação do modelo de embeddings
embeddings = OpenAIEmbeddings()

# Diretório onde o banco vetorial ficará salvo
persist_dir = "/content/rag-juridico/vectorstore"

# Se já existir, recarrega
if os.path.isdir(persist_dir):
    print("🔄 Carregando vectorstore existente...")
    vectorstore = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )

# Senão, cria do zero
else:
    print("🆕 Criando vectorstore do zero...")
    vectorstore = Chroma.from_documents(
        chunks_recursive,     # seus chunks já gerados
        embeddings,
        persist_directory=persist_dir
    )

# Salva no disco
vectorstore.persist()
print("💾 Vectorstore salvo!")



🔄 Carregando vectorstore existente...


/tmp/ipython-input-3234395558.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


💾 Vectorstore salvo!


/tmp/ipython-input-3234395558.py:30: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
!pip -q install -U langchain langchain-community langchain-text-splitters chromadb langchain-openai


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("RAG_1")

In [ ]:
# Imports e preparo
import os
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Caso ainda não tenha:
# documentos = cdc_docs + lgpd_docs
# e cada Document tem metadata["fonte"] = "cdc" ou "lgpd"

embeddings = OpenAIEmbeddings()

persist_dir = "/content/rag-juridico/vectorstore"

# Se quiser garantir que a pasta existe:
os.makedirs(persist_dir, exist_ok=True)



In [ ]:
# Separar os documentos por fonte

docs_cdc  = [d for d in documentos if d.metadata.get("fonte") == "cdc"]
docs_lgpd = [d for d in documentos if d.metadata.get("fonte") == "lgpd"]

print(f"CDC:  {len(docs_cdc)} documentos")
print(f"LGPD: {len(docs_lgpd)} documentos")


CDC:  130 documentos
LGPD: 49 documentos


In [ ]:
# Criar **duas coleções** no mesmo diretório (CDC e LGPD)
# O Chroma aceita **collection\_name**. Vamos usar o **mesmo `persist_directory`** com nomes diferentes de coleção.

# Coleção CDC
if os.path.isdir(persist_dir):
    print("🔄 Carregando/Atualizando coleção CDC...")
else:
    print("🆕 Criando diretório de persistência...")

vectorstore_cdc = Chroma(
    collection_name="cdc",
    persist_directory=persist_dir,
    embedding_function=embeddings
)

# Se a coleção ainda está vazia, vamos inserir (idempotente simples)
if docs_cdc:
    # from_documents cria do zero; para acrescentar em uma que já existe, use .add_documents
    # Aqui tentamos primeiro 'from_documents' se a coleção não existir.
    try:
        _tmp = Chroma.from_documents(
            docs_cdc,
            embeddings,
            collection_name="cdc",
            persist_directory=persist_dir,
        )
        print("✅ Coleção CDC criada a partir dos documentos.")
    except Exception:
        # Se já existe, apenas adiciona (não dá erro se IDs não forem repetidos)
        vectorstore_cdc.add_documents(docs_cdc)
        print("🔁 Documentos CDC adicionados à coleção existente.")

vectorstore_cdc.persist()

# Coleção LGPD
vectorstore_lgpd = Chroma(
    collection_name="lgpd",
    persist_directory=persist_dir,
    embedding_function=embeddings
)

if docs_lgpd:
    try:
        _tmp = Chroma.from_documents(
            docs_lgpd,
            embeddings,
            collection_name="lgpd",
            persist_directory=persist_dir,
        )
        print("✅ Coleção LGPD criada a partir dos documentos.")
    except Exception:
        vectorstore_lgpd.add_documents(docs_lgpd)
        print("🔁 Documentos LGPD adicionados à coleção existente.")

vectorstore_lgpd.persist()

print("💾 Persistência concluída!")


🔄 Carregando/Atualizando coleção CDC...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
!pip -q install -U langchain langchain-community chromadb langchain-openai

import os
os.environ["OPENAI_API_KEY"] = "RAG_1"  # não compartilhe!

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OpenAIEmbeddings()

persist_dir = "/content/rag-juridico/vectorstore"

# Recarrega/cria coleção CDC
vectorstore_cdc = Chroma(
    collection_name="cdc",
    persist_directory=persist_dir,
    embedding_function=embeddings
)
# Se for a primeira vez, popular:
# vectorstore_cdc = Chroma.from_documents(docs_cdc, embeddings, collection_name="cdc", persist_directory=persist_dir)

# Recarrega/cria coleção LGPD
vectorstore_lgpd = Chroma(
    collection_name="lgpd",
    persist_directory=persist_dir,
    embedding_function=embeddings
)
# Se for a primeira vez, popular:
# vectorstore_lgpd = Chroma.from_documents(docs_lgpd, embeddings, collection_name="lgpd", persist_directory=persist_dir)

vectorstore_cdc.persist()
vectorstore_lgpd.persist()
print("Coleções carregadas/persistidas com OpenAIEmbeddings.")


Coleções carregadas/persistidas com OpenAIEmbeddings.


In [ ]:
!pip -q install -U langchain langchain-community chromadb langchain-google-genai google-generativeai

In [ ]:
# Usar **sua chave do Google (Gemini)** com LangChain
# !pip -q install -U langchain langchain-community chromadb langchain-google-genai google-generativeai

# import os
# os.environ["GOOGLE_API_KEY"] = "OPENAI_API_KEY"

# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("RAG_1")

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

# Modelo de embedding do Google
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

persist_dir = "/content/rag-juridico/vectorstore"

# Cria/Recarrega a coleção CDC
try:
    _ = Chroma.from_documents(
        docs_cdc, embeddings,
        collection_name="cdc",
        persist_directory=persist_dir
    )
    print("✅ Coleção CDC criada (Google Embeddings).")
except Exception:
    vectorstore_cdc = Chroma(
        collection_name="cdc",
        persist_directory=persist_dir,
        embedding_function=embeddings
    )
    vectorstore_cdc.add_documents(docs_cdc)
    print("🔁 CDC adicionado à coleção existente.")

# Cria/Recarrega a coleção LGPD
try:
    _ = Chroma.from_documents(
        docs_lgpd, embeddings,
        collection_name="lgpd",
        persist_directory=persist_dir
    )
    print("✅ Coleção LGPD criada (Google Embeddings).")
except Exception:
    vectorstore_lgpd = Chroma(
        collection_name="lgpd",
        persist_directory=persist_dir,
        embedding_function=embeddings
    )
    vectorstore_lgpd.add_documents(docs_lgpd)
    print("🔁 LGPD adicionada à coleção existente.")

print("💾 Persistência concluída (Google Embeddings).")


GoogleGenerativeAIError: Error embedding content (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}

In [ ]:
# Documentos separados por fonte
docs_cdc  = [d for d in documentos if d.metadata.get("fonte") == "cdc"]
docs_lgpd = [d for d in documentos if d.metadata.get("fonte") == "lgpd"]
print(f"CDC:  {len(docs_cdc)} docs | LGPD: {len(docs_lgpd)} docs")


CDC:  130 docs | LGPD: 49 docs


In [ ]:
# Consultas segmentadas por coleção (teste)

# Reabra as coleções (útil após reiniciar a sessão)
vectorstore_cdc  = Chroma(collection_name="cdc",  persist_directory=persist_dir, embedding_function=embeddings)
vectorstore_lgpd = Chroma(collection_name="lgpd", persist_directory=persist_dir, embedding_function=embeddings)

q1 = "O que o CDC diz sobre garantia de produto com defeito?"
hits_cdc = vectorstore_cdc.similarity_search(q1, k=3)

q2 = "Quais são as bases legais previstas na LGPD?"
hits_lgpd = vectorstore_lgpd.similarity_search(q2, k=3)

print("CDC:")
for d in hits_cdc:
    print("-", d.metadata.get("fonte"), "p.", d.metadata.get("page"), "->", d.page_content[:180], "...\n")

print("LGPD:")
for d in hits_lgpd:
    print("-", d.metadata.get("fonte"), "p.", d.metadata.get("page"), "->", d.page_content[:180], "...\n")


GoogleGenerativeAIError: Error embedding content (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}

In [ ]:
# Alternativa sem OpenAI (HuggingFace)
!pip -q install -U sentence-transformers

In [ ]:
!pip -q install -U langchain langchain-community chromadb sentence-transformers

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

persist_dir = "/content/rag-juridico/vectorstore"

# Primeira criação
_ = Chroma.from_documents(
    docs_cdc, embeddings,
    collection_name="cdc",
    persist_directory=persist_dir
)
_ = Chroma.from_documents(
    docs_lgpd, embeddings,
    collection_name="lgpd",
    persist_directory=persist_dir
)

print("Coleções CDC/LGPD criadas com HuggingFaceEmbeddings.")




/tmp/ipython-input-2906386896.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Coleções CDC/LGPD criadas com HuggingFaceEmbeddings.


In [ ]:
# Consultas segmentadas

from langchain_community.vectorstores import Chroma

persist_dir = "/content/rag-juridico/vectorstore"

# Recarrega as coleções com o mesmo provider de embeddings usado na criação
vectorstore_cdc  = Chroma(collection_name="cdc",  persist_directory=persist_dir, embedding_function=embeddings)
vectorstore_lgpd = Chroma(collection_name="lgpd", persist_directory=persist_dir, embedding_function=embeddings)

q1 = "O que o CDC diz sobre garantia de produto com defeito?"
hits_cdc = vectorstore_cdc.similarity_search(q1, k=3)

q2 = "Quais são as bases legais previstas na LGPD?"
hits_lgpd = vectorstore_lgpd.similarity_search(q2, k=3)

print("CDC:")
for d in hits_cdc:
    print("-", d.metadata.get("fonte"), "p.", d.metadata.get("page"), "->", d.page_content[:180], "...\n")

print("LGPD:")
for d in hits_lgpd:
    print("-", d.metadata.get("fonte"), "p.", d.metadata.get("page"), "->", d.page_content[:180], "...\n")


CDC:
- cdc p. 2 -> Secretaria da Justiça e Cidadania
Fundação de Proteção e Defesa do Consumidor
CÓDIGO DE
PROTEÇÃO E
DEFESA DO
CONSUMIDOR
Lei nº 10.962, de 11 de outubro de 2004 
Decreto nº 2.181, d ...

- cdc p. 16 -> 15
 § 5º No caso de fornecimento de produtos in natura, 
será responsável perante o consumidor o fornecedor 
imediato, exceto quando identificado claramente seu 
produtor.
 § 6º Sã ...

- cdc p. 63 -> 62
relativa ao preço à vista do produto, suas características 
e código.
Art. 2º-A Na venda a varejo de produtos fraciona -
dos em pequenas quantidades, o comerciante deverá 
infor ...

LGPD:
- lgpd p. 34 -> 35Lei no 13.709/2018
e) tenha o objetivo de estabelecer relação de confiança com o titular, 
por meio de atuação transparente e que assegure mecanismos de partici-
pação do titular ...

- lgpd p. 48 -> Este livro traz ao leitor a Lei Geral de Proteção de Dados Pessoais (LGPD), Lei 
no 13.709/2018. Tendo por objeto o tratamento de dados pessoais, inclusive 
nos meios di

## **5 — Recuperação Semântica**

### **Descrição**

Avaliar como o banco vetorial funciona **antes** de envolver um modelo de linguagem (LLM).  
O objetivo é garantir que os *chunks* retornados são relevantes e pertencem à lei correta.

***

### **Objetivo**

Testar a busca vetorial utilizando um *retriever* para verificar:

*   quais *chunks* são recuperados
*   se eles vêm da fonte correta (CDC ou LGPD)
*   se os metadados estão sendo preservados
*   se o banco vetorial está funcionando conforme esperado

***

### **Tarefas**

1.  **Criar um retriever**
    *   Configurar `k = 5` (retornar os 5 chunks mais relevantes)

2.  **Realizar buscas com perguntas jurídicas**, por exemplo:
    *   “O fornecedor pode se eximir de responsabilidade?”
    *   “Em que casos o consentimento é obrigatório?”

3.  **Exibir apenas**:
    *   o texto dos chunks recuperados
    *   seus metadados (ex.: `fonte`, `page`, etc.)

***

### **Reflexão**

*   Os chunks recuperados pertencem à lei correta?
    *   Perguntas sobre responsabilidade civil devem retornar **CDC**.
    *   Perguntas sobre consentimento e tratamento de dados devem retornar **LGPD**.
*   A segmentação (chunking) está ajudando ou atrapalhando a recuperação?
*   Os metadados estão preservados e corretos?

In [ ]:
# Recuperação semântica (sem LLM)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
retriever = vectorstore_cdc.as_retriever(search_kwargs={"k": 5})

In [ ]:
docs_recuperados = retriever.invoke(pergunta)

NameError: name 'pergunta' is not defined

In [ ]:
# Recuperação semântica (sem LLM)
retriever = vectorstore_cdc.as_retriever(search_kwargs={"k": 5})

# Pergunta de teste
pergunta = "O fornecedor pode se eximir de responsabilidade?"

# Método CORRETO:
docs_recuperados = retriever.invoke(pergunta)

# Exibição dos chunks recuperados
for i, doc in enumerate(docs_recuperados, 1):
    print(f"\n--- Chunk {i} ---")
    print(f"Fonte: {doc.metadata.get('fonte')}")
    print(f"Página: {doc.metadata.get('page')}")
    print(doc.page_content[:500], "...")


--- Chunk 1 ---
Fonte: cdc
Página: 29
28
 I - impossibilitem, exonerem ou atenuem a respon -
sabilidade do fornecedor por vícios de qualquer natu -
reza dos produtos e serviços ou impliquem renúncia ou 
disposição de direitos. Nas relações de consumo entre 
o fornecedor e o consumidor-pessoa jurídica, a indeni-
zação poderá ser limitada, em situações justificáveis;
 II - subtraiam ao consumidor a opção de reembolso 
da quantia já paga, nos casos previstos neste Código;
 III - transfiram responsabilidades a terceiros;
 IV - estabeleç ...

--- Chunk 2 ---
Fonte: cdc
Página: 17
16
 Art. 20. O fornecedor de serviços responde pelos 
vícios de qualidade que os tornem impróprios ao consu-
mo ou lhes diminuam o valor, assim como por aqueles 
decorrentes da disparidade com as indicações cons -
tantes da oferta ou mensagem publicitária, podendo o 
consumidor exigir, alternativamente e à sua escolha:
 I - a reexecução dos serviços, sem custo adicional 
e quando cabível;
 II - a restituição imedi

In [ ]:
# Recuperação semântica (sem LLM)
retriever = vectorstore_cdc.as_retriever(search_kwargs={"k": 5})

# Pergunta de teste
pergunta = "Em que casos o consentimento é obrigatório?"

# Método CORRETO:
docs_recuperados = retriever.invoke(pergunta)

# Exibição dos chunks recuperados
for i, doc in enumerate(docs_recuperados, 1):
    print(f"\n--- Chunk {i} ---")
    print(f"Fonte: {doc.metadata.get('fonte')}")
    print(f"Página: {doc.metadata.get('page')}")
    print(doc.page_content[:500], "...")


--- Chunk 1 ---
Fonte: cdc
Página: 18
17
serão as pessoas jurídicas compelidas a cumpri-las e 
a reparar os danos causados, na forma prevista neste 
Código.
 Art. 23. A ignorância do fornecedor sobre os vícios 
de qualidade por inadequação dos produtos e serviços 
não o exime de responsabilidade.
 Art. 24. A garantia legal de adequação do produto ou 
serviço independe de termo expresso, vedada a exone-
ração contratual do fornecedor.
 Art. 25. É vedada a estipulação contratual de cláusu-
la que impossibilite, exonere ou atenue a obr ...

--- Chunk 2 ---
Fonte: cdc
Página: 79
78
§ 3º As penalidades previstas nos incisos III a XI 
deste artigo sujeitam-se a   posterior confirmação pelo 
órgão normativo ou regulador da atividade, nos limites 
de sua competência.
Art. 19. Toda pessoa física ou jurídica que fizer ou 
promover publicidade enganosa ou abusiva ficará sujeita 
à pena de multa, cumulada com aquelas previstas no 
artigo anterior, sem prejuízo da competência de outros 
órgãos adm

In [ ]:
print("\n=== Consulta no CDC ===")
retriever_cdc = vectorstore_cdc.as_retriever(search_kwargs={"k": 5})
for d in retriever_cdc.invoke(pergunta):
    print(d.metadata, d.page_content[:200], "...")

print("\n=== Consulta na LGPD ===")
retriever_lgpd = vectorstore_lgpd.as_retriever(search_kwargs={"k": 5})
for d in retriever_lgpd.invoke(pergunta):
    print(d.metadata, d.page_content[:200], "...")


=== Consulta no CDC ===
{'fonte': 'cdc', 'total_pages': 130, 'producer': 'Adobe PDF Library 17.0', 'page': 18, 'source': '/content/rag-juridico/dados/cdc.pdf', 'moddate': '2025-02-11T11:13:18-03:00', 'trapped': '/False', 'page_label': '19', 'creationdate': '2025-02-11T11:13:18-03:00', 'creator': 'Adobe InDesign 20.1 (Windows)', 'title': 'CDC_capa_2025.indd'} 17
serão as pessoas jurídicas compelidas a cumpri-las e 
a reparar os danos causados, na forma prevista neste 
Código.
 Art. 23. A ignorância do fornecedor sobre os vícios 
de qualidade por inadequaçã ...
{'creator': 'Adobe InDesign 20.1 (Windows)', 'trapped': '/False', 'moddate': '2025-02-11T11:13:18-03:00', 'page_label': '80', 'producer': 'Adobe PDF Library 17.0', 'total_pages': 130, 'creationdate': '2025-02-11T11:13:18-03:00', 'fonte': 'cdc', 'source': '/content/rag-juridico/dados/cdc.pdf', 'title': 'CDC_capa_2025.indd', 'page': 79} 78
§ 3º As penalidades previstas nos incisos III a XI 
deste artigo sujeitam-se a   posterior co

# 6 – Primeira versão de RAG

***

## Descrição

Primeira implementação combinando *retrieval* + *LLM* para responder perguntas com base em documentos do CDC e da LGPD.

***

## Objetivo

Integrar *retrieval* com LLM garantindo que o modelo:

**Responda somente com base no contexto fornecido.**

***

## Tarefas

### 1. Criar o prompt-base

```markdown
Responda somente com base no contexto fornecido.
Se a resposta não estiver no contexto, diga que não encontrou informações suficientes.
```

### 2. Injetar os chunks recuperados como contexto

Exemplo de estrutura:

```markdown
### Contexto
{{context}}

### Pergunta
{{question}}

### Instruções
Responda somente com base no contexto acima.
```

### 3. Testar com perguntas

*   “O consumidor pode desistir da compra feita pela internet?”
*   “Quais são os direitos do titular de dados pessoais?”

***

## Resultado esperado

*   Respostas corretas e fundamentadas.
*   Exibição das fontes (por exemplo: CDC art. 49, LGPD art. 18).

Exemplo:

```markdown
### Fontes consultadas:
- CDC – Artigo 49
- LGPD – Artigo 18
```

***

## Reflexão

E se a pergunta não estiver relacionada ao CDC ou à LGPD?

Seu RAG deve estar preparado para:

*   Detectar ausência de contexto relevante
*   Evitar alucinações
*   Retornar algo como:

```markdown
A pergunta não está relacionada às fontes fornecidas (CDC e LGPD).
Não encontrei informações suficientes no contexto.
```

In [ ]:
!pip -q install --upgrade langchain langchain-openai faiss-cpu tiktoken pydantic==2.9.2

In [ ]:
!pip -q install -U "langchain>=0.2" "langchain-openai>=0.1" "langchain-community>=0.2" \
                 "langchain-text-splitters>=0.2" faiss-cpu tiktoken pydantic==2.9.2


In [ ]:

!pip -q install -U langchain langchain-community chromadb sentence-transformers

import os

# Coloque sua chave (ou use secrets do Colab)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage



Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
!pip -q install -U "langchain>=0.2" "langchain-community>=0.2" \
                 "langchain-text-splitters>=0.2" chromadb faiss-cpu \
                 sentence-transformers tiktoken pydantic==2.9.2

In [ ]:
# Imports e Configuração Chroma + Embeddings

import os
import warnings
warnings.filterwarnings("ignore")

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI  # LLM para responder (pode ser gpt-3.5-turbo, gpt-4o-mini, etc.)

# Se quiser GPU no Colab: Runtime > Change runtime type > GPU
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Diretório de persistência do Chroma
PERSIST_DIR = "chroma_cdc_lgpd"

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
# Ingestão de documentos CDC/LGPD
cdc_text = """
CDC - Artigo 49: O consumidor pode desistir do contrato, no prazo de 7 dias a contar de sua assinatura
ou do ato de recebimento do produto ou serviço, sempre que a contratação ocorrer fora do estabelecimento comercial...
"""

lgpd_text = """
LGPD - Artigo 18: O titular dos dados pessoais tem direito a obter do controlador, a qualquer momento e mediante requisição:
I - confirmação da existência de tratamento; II - acesso aos dados; III - correção de dados incompletos, inexatos ou desatualizados; ...
"""

docs_raw = [
    Document(page_content=cdc_text.strip(), metadata={"fonte": "CDC - Art. 49"}),
    Document(page_content=lgpd_text.strip(), metadata={"fonte": "LGPD - Art. 18"}),
]

# Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=150)
docs = splitter.split_documents(docs_raw)
len(docs), docs[0].metadata



(2, {'fonte': 'CDC - Art. 49'})

In [ ]:
# Criar/Atualizar o índice **Chroma** e o **Retriever**

# Cria (ou atualiza) um repositório Chroma com persistência
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=PERSIST_DIR
)
vectordb.persist()

# Reabrir depois em outra sessão/execução:
# vectordb = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)

retriever = vectordb.as_retriever(search_kwargs={"k": 4})


In [ ]:
# Prompt e Função `responder(...)` (com fontes e anti-alucinação)

# Defina sua chave se for usar modelos OpenAI via ChatOpenAI
# (se não tiver, troque o ChatOpenAI por outro LLM que você use local/endpoint próprio)
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# os.environ["OPENAI_API_KEY"] = os.environ.get("RAG-1")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # ou "gpt-4o-mini" para melhor qualidade/custo

PROMPT_TMPL = """
Você é um assistente jurídico que responde exclusivamente com base no CONTEXTO.
Se a resposta não estiver no contexto, diga explicitamente que não há informações suficientes no contexto.
Responda de forma objetiva e cite as fontes ao final.

[CONTEXTO]
{context}

[PERGUNTA]
{question}

[INSTRUÇÕES]
- Não invente informações fora do contexto.
- Se a pergunta não for sobre CDC/LGPD ou não houver contexto relevante, explique que não encontrou base no contexto.
- Cite as fontes pelo campo 'fonte' dos trechos utilizados.
"""

prompt = PromptTemplate.from_template(PROMPT_TMPL)

def _format_context_with_sources(docs):
    blocks = []
    for d in docs:
        fonte = d.metadata.get("fonte", "Fonte desconhecida")
        blocks.append(f"[Fonte: {fonte}]\n{d.page_content}")
    return "\n\n---\n\n".join(blocks)

def responder(pergunta: str):
    docs_to_use = retriever.get_relevant_documents(pergunta)
    if not docs_to_use:
        return {
            "answer": "Não encontrei informações suficientes no contexto para responder à pergunta.",
            "sources": []
        }

    context = _format_context_with_sources(docs_to_use)
    final_prompt = prompt.format(context=context, question=pergunta)
    resp = llm([HumanMessage(content=final_prompt)])

    sources = list(dict.fromkeys([d.metadata.get("fonte", "Fonte desconhecida") for d in docs_to_use]))
    return {"answer": resp.content, "sources": sources}


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
# 6 Testes
q1 = "O consumidor pode desistir da compra feita pela internet?"
q2 = "Quais são os direitos do titular de dados pessoais?"
q3 = "Como calcular ICMS de energia elétrica?"  # fora de CDC/LGPD para testar a negativa controlada

for q in (q1, q2, q3):
    out = responder(q)
    print("PERGUNTA:", q)
    print("RESPOSTA:\n", out["answer"])
    print("FONTES:", ", ".join(out["sources"]) if out["sources"] else "—")
    print("="*80)


AttributeError: 'VectorStoreRetriever' object has no attribute 'get_relevant_documents'

In [ ]:

from typing import List, Dict
from langchain_core.documents import Document

def _format_context_with_sources(docs: List[Document]) -> str:
    blocks = []
    for d in docs:
        fonte = d.metadata.get("fonte", "Fonte desconhecida")
        blocks.append(f"[Fonte: {fonte}]\n{d.page_content}")
    return "\n\n---\n\n".join(blocks)

def responder(pergunta: str) -> Dict[str, str]:
    # ✔️ Versão compatível com langchain>=0.2
    docs_to_use = retriever.invoke(pergunta)  # <- trocado de get_relevant_documents para invoke

    if not docs_to_use:
        return {
            "answer": "Não encontrei informações suficientes no contexto para responder à pergunta.",
            "sources": []
        }

    context = _format_context_with_sources(docs_to_use)
    final_prompt = prompt.format(context=context, question=pergunta)

    resp = llm([HumanMessage(content=final_prompt)])
    sources = list(dict.fromkeys(d.metadata.get("fonte", "Fonte desconhecida") for d in docs_to_use))

    return {"answer": resp.content, "sources": sources}


In [ ]:
    # Assíncrona (útil quando você for paralelizar perguntas):
    docs_to_use = await retriever.ainvoke(pergunta)

   # Lote (várias perguntas de uma vez):

    perguntas = ["Q1", "Q2", "Q3"]
    lista_docs = retriever.batch(perguntas)  # retorna uma lista; cada item é a lista de docs p/ a pergunta correspondente




In [ ]:

def responder(pergunta: str):
    # Recupera documentos via retriever (novo padrão LC 0.2+)
    docs_to_use = retriever.invoke(pergunta)

    if not docs_to_use:
        return {
            "answer": "Não encontrei informações suficientes no contexto para responder à pergunta.",
            "sources": []
        }

    context = _format_context_with_sources(docs_to_use)
    final_prompt = prompt.format(context=context, question=pergunta)

    # Chamada correta do LLM
    resp = llm.invoke([HumanMessage(content=final_prompt)])

    sources = list(dict.fromkeys(
        d.metadata.get("fonte", "Fonte desconhecida") for d in docs_to_use
    ))

    return {"answer": resp.content, "sources": sources}


In [ ]:

q1 = "O consumidor pode desistir da compra feita pela internet?"
q2 = "Quais são os direitos do titular de dados pessoais?"
q3 = "Como calcular ICMS de energia elétrica?"  # fora do CDC/LGPD p/ testar a negativa

for q in (q1, q2, q3):
    out = responder(q)
    print("PERGUNTA:", q)
    print("RESPOSTA:\n", out["answer"])
    print("FONTES:", ", ".join(out["sources"]) if out["sources"] else "—")
    print("="*80)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

7 – Reranking com LLM

***

## Descrição

Nesta etapa, vamos **melhorar a qualidade dos trechos (chunks) selecionados** para o RAG usando um mecanismo de **reranking baseado em LLM**.  
A ideia é: primeiro recuperar muitos chunks, depois pedir para o LLM pontuar e selecionar apenas os mais relevantes.

***

## Objetivo

Aprimorar o contexto entregue ao modelo final, reduzindo ruído e aumentando precisão, especialmente quando muitos trechos são semelhantes ou parcialmente relevantes.

***

## Tarefas

### 1. Recuperar mais chunks inicialmente

*   Use `k = 15` na busca vetorial inicial.  
    Isso aumenta a cobertura e traz opções para o reranker escolher.

### 2. Usar o LLM para dar uma nota de relevância

Construa um prompt para que o LLM avalie **cada chunk**, atribuindo uma nota de **0 a 10** para relevância em relação à pergunta.

Modelo do prompt:

```markdown
Avalie a relevância do trecho abaixo para responder à pergunta dada.

Pergunta:
{pergunta}

Trecho:
{chunk}

Responda apenas com um número de 0 a 10 indicando o quanto esse trecho é útil para responder à pergunta.
```

### 3. Selecionar apenas os 4 melhores chunks

*   Ordene os 15 chunks pela nota do LLM.
*   Mantenha apenas os **4 mais relevantes**.

### 4. Usar os 4 melhores no prompt final

*   Monte o contexto final **somente** a partir desses 4 chunks reranqueados.
*   Execute o LLM para gerar a resposta completa com base nesses trechos.

***

## Comparação

Execute a mesma pergunta em dois cenários:

1.  **Sem reranking**  
    – Pipeline padrão: o retriever já escolhe k=4 diretamente.

2.  **Com reranking LLM**  
    – Recupera 15 → reranking → escolhe 4 melhores → resposta final.

Avalie:

*   Clareza
*   Precisão
*   Citação correta das fontes
*   Menos ruído ou desvios de contexto

***

## Resultado esperado

*   Respostas mais consistentes quando há muitos trechos similares.
*   Redução de alucinações.
*   Seleção de contexto mais relevante.
*   Melhor explicação e fundamentação.


In [ ]:
# Reranking com LLM

retriever_15 = vectorstore.as_retriever(search_kwargs={"k": 15})


In [ ]:
# Função de reranking
def rerank(pergunta, docs):
    avaliados = []

    for doc in docs:
        prompt = f"""
Dê uma nota de 0 a 10 para a relevância do texto abaixo
em relação à pergunta.

Pergunta:
{pergunta}

Texto:
{doc.page_content}

Responda apenas com um número.
"""
        nota = llm([HumanMessage(content=prompt)]).content
        try:
            nota = float(nota)
        except:
            nota = 0
        avaliados.append((nota, doc))

    avaliados.sort(reverse=True, key=lambda x: x[0])
    return [doc for _, doc in avaliados[:4]]



In [ ]:
# RAG com reranking
def responder_com_rerank(pergunta):
    docs_iniciais = retriever_15.get_relevant_documents(pergunta)
    docs_finais = rerank(pergunta, docs_iniciais)

    contexto = "\n\n".join(
        f"[Fonte: {d.metadata['fonte']}]\n{d.page_content}"
        for d in docs_finais
    )

    prompt = f"""
Responda SOMENTE com base no contexto abaixo.

Contexto:
{contexto}

Pergunta:
{pergunta}
"""

    resposta = llm([HumanMessage(content=prompt)])
    return resposta.content



In [ ]:
#  Comparação final
print("Sem reranking:\n", responder("O consumidor pode desistir da compra feita pela internet?"))
print("\nCom reranking:\n", responder_com_rerank("O consumidor pode desistir da compra feita pela internet?"))


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}